In [1]:
%load_ext autoreload
% autoreload 2
import reinforcement_expm
import numpy as np
import tensorflow as tf
from agents import load_expm_params
import copy

//anaconda/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
expm_name = 'LunarLander_Soft'
algo = 'DQN'
params = load_expm_params(algo,expm_name)

test_case_params = {
    'soft' : dict(agent = dict(soft_learning = True)),
#     'dueling/boltz' : dict(agent = dict(soft_learning = False,dueling =True), policy = dict(action_choice = 'Boltzmann')),
#     'boltz' : dict(agent = dict(soft_learning = False,dueling =False), policy = dict(action_choice = 'Boltzmann')),
#     'epsilon' : dict(agent = dict(soft_learning = False,dueling =True), policy = dict(action_choice = 'Epsilon'))
}

test_case_evaluation(params,test_case_params,runs_per_point = 2)

# grid_params = dict(
# lr = [1e-3,1e-4],
# reward_scale = [1.0,0.1,10.0]
# )

# grid_search(params,grid_params)
    

NameError: name 'test_case_evaluation' is not defined

In [4]:
from matplotlib import pyplot as plt
%matplotlib inline

def grid_search(params,grid_params,runs_per_point = 3):
    
    to_test = grid_params.items()
    to_test_keys = [val[0] for val in to_test]
    test_pts = np.meshgrid(*[val[1] for val in to_test])
    test_pts = np.reshape(np.transpose(test_pts),[-1,len(test_pts)])

    
    max_episodes = params['runner']['max_episodes']

    rewards = np.zeros((len(test_pts),runs_per_point,max_episodes))
    for i,tp in enumerate(test_pts):
        test_params = copy.deepcopy(params)
        
        for key,val in zip(to_test_keys,tp):
            test_params['agent'][key]= val
            print('Testing %s: %.2e' % (key,val))
        for j in range(runs_per_point):
            runner = reinforcement_expm.run_expm(expm_name,algo, params = test_params)
            rewards[i,j] = runner.episode_rewards

    title = ', '.join(to_test_keys)
    labels = [', '.join(["%.2e" % val for val in pt]) for pt in test_pts]

    x = range(np.shape(rewards)[-1])
    ys = np.mean(rewards,axis=1)
    errors = np.std(rewards,axis=1)

    for y,error,lr in zip(ys,errors,labels):
        plt.plot(x, y, '-',label=lr)
        plt.fill_between(x, y-error, y+error,alpha=0.2)
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.legend()
    plt.show()
    plt.close()

In [3]:
def test_case_evaluation(params,test_case_params,runs_per_point = 3):
    
    runs_per_point = 3
    max_episodes = params['runner']['max_episodes']
    rewards = np.zeros((len(test_case_params),runs_per_point,max_episodes))
    
    for i,(test_name,test) in enumerate(iter(test_case_params.items())):
        test_params = copy.deepcopy(params)
        for cat, items in iter(test.items()):
            for item, values in iter(items.items()):
                test_params[cat][item] = values

        print('Testing %s' % (test_name))
        print(test_params)
        
        for j in range(runs_per_point):
            runner = reinforcement_expm.run_expm(expm_name,algo, params = test_params)
            rewards[i,j] = runner.episode_rewards

    labels = test_case_params.keys()

    x = range(np.shape(rewards)[-1])
    ys = np.mean(rewards,axis=1)
    errors = np.std(rewards,axis=1)

    for y,error,lr in zip(ys,errors,labels):
        plt.plot(x, y, '-',label=lr)
        plt.fill_between(x, y-error, y+error,alpha=0.2)
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.legend()
    plt.show()
    plt.close()